In [ ]:
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 68.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00


In [ ]:
pip install nltk

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load dataset from Hugging Face
dataset = load_dataset("multi_news")

# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)

# Select subsets for training, validation, and testing
train_set = df.iloc[:16000]
validation_set = df.iloc[16000:18000]
test_set = df.iloc[18000:20000]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [ ]:
import re

def clean_data(dataset):
    cleaned_texts = []
    for text in dataset['document']:
        # Ensure the example is a string
        if isinstance(text, str):
            # Remove special characters
            cleaned_text = re.sub(r'[^\w\s]', '', text)
            cleaned_text = re.sub(r'[@|#|_!]', '', text)
            cleaned_text = re.sub(r'http\S+', '', text)
            cleaned_text = re.sub("\n+", "\n",  text)
            cleaned_text = re.sub(" +", " ",  text)
            cleaned_text = re.sub("\t+", "\t",  text)
            cleaned_text = text.lower()
            cleaned_texts.append(cleaned_text)
        else:
            cleaned_texts.append("")  # If it's not a string, replace with an empty string
    cleaned_dataset = dataset.copy()
    cleaned_dataset['document'] = cleaned_texts
    return cleaned_dataset

# Clean the train_set
cleaned_train_set = clean_data(train_set)

# Clean the validation_set
cleaned_validation_set = clean_data(validation_set)

# Clean the test_set
cleaned_test_set = clean_data(test_set)

In [ ]:
# Create a new dictionary to hold the cleaned datasets
cleaned_datasets = {}

# Add the cleaned and split datasets to the dictionary
cleaned_datasets['train'] = cleaned_train_set
cleaned_datasets['validation'] = cleaned_validation_set
cleaned_datasets['test'] = cleaned_test_set

# Convert the dictionary into a single pandas DataFrame
cleaned_df = pd.concat(cleaned_datasets.values(), keys=cleaned_datasets.keys())

# Reset index
cleaned_df = cleaned_df.reset_index(level=0).rename(columns={'level_0': 'dataset'})

# Reset index for the DataFrame
cleaned_df = cleaned_df.reset_index(drop=True)

# Display the new dataset
print(cleaned_df.head())

  dataset                                           document  \
0   train  emergency responders work at the scene of a cr...   
1   train  signs of aging have been partially reversed in...   
2   train  for texas rangers catcher mike napoli not to w...   
3   train  enlarge the mandarin oriental at time warner c...   
4   train  use of this tool is open to all members of red...   

                                             summary  
0  – The cause of Sunday morning's deadly train c...  
1  – In the future, we may be able to treat aging...  
2  – Nobody loves working pitching matchups like ...  
3  – A highly regarded tech investor was found de...  
4  – It all started back in 2011 when a Reddit ac...  


In [ ]:
from datasets import DatasetDict, Dataset

# Assuming you already have your cleaned and split datasets:
# cleaned_train_set, cleaned_validation_set, cleaned_test_set

# Convert the cleaned pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(cleaned_train_set)
validation_dataset = Dataset.from_pandas(cleaned_validation_set)
test_dataset = Dataset.from_pandas(cleaned_test_set)

# Create a DatasetDict containing the datasets
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

# Print the DatasetDict information
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 2000
    })
})


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "google/mt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

In [ ]:
import nltk

In [ ]:
pip install https://huggingface.co/spacy/en_core_web_sm/resolve/main/en_core_web_sm-any-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 90.4 MB/s eta 0:00:00


In [ ]:
!pip install https://huggingface.co/spacy/en_core_web_lg/resolve/main/en_core_web_lg-any-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.5 MB/s eta 0:00:00


In [ ]:
import spacy

nlp_sm = spacy.load("en_core_web_sm")
nlp_lg = spacy.load('en_core_web_lg')

def create_word_counts(raw_text, word_count_dict_input=None):
    #"""
    #Takes a raw text file, tokenizes and lemmatizes it,
    #and counts the individual lemmas. Returns a dictionary
    #with word counts.
    #"""

    doc = nlp_lg(raw_text)

    if word_count_dict_input is None:
        word_count_dict = {}
    else:
        word_count_dict = word_count_dict_input

    for token in doc:
        if not token.is_stop:
            word_lemma = token.lemma_
            current_count = word_count_dict.get(word_lemma, 0)
            current_count += 1
            word_count_dict[word_lemma] = current_count

    return word_count_dict


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
def preprocess_function(dataset_dict):
    text_token_len = 512
    sum_token_len = 84
    input_text = dataset_dict["document"]
    sum_text = dataset_dict["summary"]
    text_token = tokenizer(input_text, truncation=True, padding=True, max_length=text_token_len)
    sum_token = tokenizer(sum_text, truncation=True, padding=True, max_length=sum_token_len)
    #num_of_words = sum(len(text.split()) for text in sum_texts)
    model_inputs = {
        "tokenized_input_ids": text_token.input_ids,
        "tokenzied_target_ids": sum_token.input_ids,
        #"number_of_words": num_of_words,
    }
    return model_inputs

In [ ]:
tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'tokenized_input_ids', 'tokenzied_target_ids'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['document', 'summary', 'tokenized_input_ids', 'tokenzied_target_ids'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['document', 'summary', 'tokenized_input_ids', 'tokenzied_target_ids'],
        num_rows: 2000
    })
})

In [ ]:
# Convert train_dataset to a pandas DataFrame  #NO USE
train_df = train_dataset.to_pandas()

# Display train_df as a table
print(train_df)


                                                document  \
0      emergency responders work at the scene of a cr...   
1      signs of aging have been partially reversed in...   
2      for texas rangers catcher mike napoli not to w...   
3      enlarge the mandarin oriental at time warner c...   
4      use of this tool is open to all members of red...   
...                                                  ...   
15995  the titan arum (amorphophallus titanum), also ...   
15996  “instead of trying to sort out with reason and...   
15997  a ]wave of fetish-related google searches foll...   
15998  story highlights the fbi says four similar let...   
15999  police officers gather on a dirt path at kenil...   

                                                 summary  
0      – The cause of Sunday morning's deadly train c...  
1      – In the future, we may be able to treat aging...  
2      – Nobody loves working pitching matchups like ...  
3      – A highly regarded tech investor wa

In [ ]:
print(tokenized_datasets['train'][1]['document'])

signs of aging have been partially reversed in mice, according to research led by scientists at the salk institute for biological studies. the study indicates that rejuvenation, long considered science fiction, may one day become reality. 
 
 mice genetically engineered to replicate progeria, a disease of accelerated aging, lived 30 percent longer after receiving the treatment. and normally aged mice experienced improvement in pancreatic and muscle function, along with better wound healing. 
 
 the results suggest that aging in humans may likewise be a plastic process that can be controlled, said noted stem cell scientist juan carlos izpisúa belmonte, who led the study along with salk institute colleague alejandro ocampo. 
 
 the study, published thursday in the journal cell, is a proof of principle, the scientists cautioned. years of research lie ahead before any therapy can be tried in people. the complete study can be found at j.mp/belmonteaging. 
 
 the researchers accomplished thi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/T5

/content/drive/MyDrive/T5


In [ ]:
pip install torch

In [ ]:
import torch
model_save_name = 't5model2800.pt'
path = F"/content/drive/MyDrive/T5/{model_save_name}"
#torch.save(model.state_dict(), path)

torch.save(model.state_dict(), path)

In [ ]:

from datasets import load_metric
from transformers import TrainingArguments, Trainer
import tensorflow as tf
import torch
import nltk

nltk.download('wordnet')
#meteor_metric = load_metric("meteor")

#def compute_metrics(eval_pred):
    #predictions, labels = eval_pred
    #predictions = torch.argmax(predictions, axis=-1)
    #predictions = [tokenizer.decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=True) for pred in predictions]
    #references = [tokenizer.decode(ref, skip_special_tokens=True, clean_up_tokenization_spaces=True) for ref in labels]
    #meteor_score = meteor_metric.compute(predictions=predictions, references=references)
    #return {"meteor": meteor_score["meteor"]}

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=5,
    save_strategy="epoch",
    save_steps=500,
    num_train_epochs=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    #compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)
trainer.train()



#eval_results = trainer.evaluate()
#print("Evaluation results:", eval_results)

[nltk_data] Downloading package wordnet to /root/nltk_data...
The following columns in the training set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: document, summary. If document, summary are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16000
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 8000


Epoch,Training Loss,Validation Loss
1,3.559400,2.701250
2,3.059000,2.671459


The following columns in the evaluation set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: document, summary. If document, summary are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-4000
Configuration saved in ./results/checkpoint-4000/config.json
Model weights saved in ./results/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-4000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-4000/special_tokens_map.json
Copy vocab file to ./results/checkpoint-4000/spiece.model
The following columns in the evaluation set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: document, summary. If document, summary are not expected by `MT5ForConditionalGeneration.forward`,  you can

TrainOutput(global_step=8000, training_loss=3.3276130393743517, metrics={'train_runtime': 7934.1971, 'train_samples_per_second': 4.033, 'train_steps_per_second': 1.008, 'total_flos': 3.8369476214784e+16, 'train_loss': 3.3276130393743517, 'epoch': 2.0})